<a href="https://colab.research.google.com/github/InhyeokYoo/Pytorch-study/blob/master/7%EC%9E%A5%20%ED%95%99%EC%8A%B5%20%EC%8B%9C%20%EC%83%9D%EA%B8%B8%20%EC%88%98%20%EC%9E%88%EB%8A%94%20%EB%AC%B8%EC%A0%9C%EC%A0%90/2.%20Weight_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weight Regularization
- 최적화 함수의 weight decay로 강도를 조절할 수 있다
- ```torch.optim.SGD(params, lr=1, momentum=0, dampening=0, weight_decay=0, nesterov=False)```
- 모델이 오버피팅될 경우, 적절한 regularization을 통해 이를 극복할 수 있음.
- 정형화 부분을 빼고는 CNN 코드와 동일함.

## Parameter Norm Penalties
많은 regularization 방법은 parrameter norm penalty인 $\Omega(\theta)$를 objective function $J$에 추가하여 모델의 수용력을 제한시키는 방법에 기반한다. 이러한 regularized ojbective function을 $\tilde J$를 다음과 같이 표현한다:
$$
\tilde J(\theta; X, y) = J(\theta; X, y) + \alpha\Omega(\theta)
$$
where $\alpha \in [0, \infty)$  is a hyperparamter that weights the relative contribution of the norm penealty, $\Omega$, relative to the standard objective funcion J.  
이에 대해 더 자세하게 탐구하기 전에 한가지 알아야 할 것은 neural network에서 paramter norm penalty $\Omega$는 각 layer에서 affine transformation의 *weight만을* penalize하고 bias는 regularize 하지 않는다는 점이다. Bias는 정확하게 fitting하기 위해 weight에 비해 덜 많은 데이터를 필요로 한다. 각 weight는 두 변수가 어떠한 관계가 있는지를 파악한다. 반면 각 bias는 오직 하나의 variable만을 컨트롤한다. 즉, 이는 biase는 unregularize한 상태로 내버려둔채로 너무 많은 variance를 유도하지 않는다는 뜻이다. 또한,  bias parameter를 regularizing하면 과도한 underfitting이 발생할 수 있다. 그러므로, vector $w$를 이용하여 norm penalty에 영향을 끼치는 weight를 표현하고, $\theta$는 모든 parameter를 표현한다.  
Neural network에서 각 layer마다 다른 coefficient $\alpha$를 통해 분리된 penalty를 사용하는 것이 바람직하다. 그러나 이는 여러 hyperparameter의 정확한 값을 찾는데 소모되는 비용이 생기기 때문에, search space의 크기를 줄이기 위해서 같은 weight decay를 사용하는 것도 합리적인 선택이다.

### 7.1.1 L2 Parameter Regularization
L2 norm은 일반적으로 wegith decay라고도 알려져있다. 이 regularization 전략은 regularization term $\Omega(\theta)=\frac1 2 ||w||^2_2$를 objective function에 더하여 weights를 원점(origin)에 가깝게 유도하는 것이다. 좀 더 간단하게 표현하면, bias parameter는 없다고 가정하므로, $\theta$는 그냥 $w$가 된다. 이러한 모델은 다음과 같은 total objective function을 갖는다:
$$
\tilde J(w; X, y) = \frac\alpha 2 w^Tw + J(w; X, y),
$$  
그리고 다음에 대응되는 parameter gradient를 갖는다  
$$
\nabla_w\tilde J(w; X, y) = \alpha w + \nabla_w J(w;X,y).
$$  
weight를 업데이트 하기 위해 하나의 gradient step만 취할 경우:  
$$
w \gets w - \epsilon(\alpha w + \nabla _w J(w; X, y)).
$$  
다른 방식으로 표현하면 업데이트는 다음과 같다:
$$
w \gets (1-\epsilon\alpha)w - \epsilon\nabla_wJ(w; X, y).
$$  

보다시피 weight decay term의 덧셈이 learning rule을 수정하였다. 일반적인 gradient step을 수행하기전에, 매 step에서 constant factor($\alpha$)의 곱셈을 통한 weight vector를 수렴시킨다. 

# Regularization
"Any modification we make to a learning algorithm that is intended to **reduce its generalization error** but **not its training error**"  

Deep learning의 맥락에서, 대부분의 regularization 전략은 estimators를 regularizing 하는데 초점이 맞춰져 있다. 즉, bias를 높히고, variance를 낮추는 것이 그 목적이다.

# 1. Setting

## 1) Import required libraries

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

## 2) Set Hyperparameters

In [0]:
batch_size = 256
learning_rate = 0.0002
num_epochs = 10

# 2. Data

In [0]:
# Download data

# transforms.ToTensor() -> PIL image나 numpy형식을 torch.tensor로 바꾸어줌.
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 21003748.08it/s]                            


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


32768it [00:00, 329694.18it/s]
0it [00:00, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


1654784it [00:00, 5346402.12it/s]                           
8192it [00:00, 128560.56it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw
Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [0]:
# Check Dataset

# 정보를 확인할 수 있음.
print(mnist_train)
print('='*100)

# mnist_train의 element들은 data와 label의 tuple로 이루어져있고, 60000개가 있음.
# print(mnist_train[0])

# 데이터는 [1x28x28]
print(mnist_train[0][0].size())
# label은 integer
print(mnist_test[0][1])

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train
    StandardTransform
Transform: ToTensor()
torch.Size([1, 28, 28])
7


In [0]:
# Set dataloader

train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

# 3. Model & Optimizer

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            # CNN(input_channel, output_channel, kernel_size)
            nn.Conv2d(1, 16, 3, padding=1), # 28 x 28 -> 30 x 30 (padding) -> 28 x 28
            nn.ReLU(True),
            nn.Conv2d(16, 32, 3, padding=1), # 28 x 28 -> 30 x 30 (padding) -> 28 x 28
            nn.ReLU(True),

            # MaxPool2d(kernel_size, stride)
            nn.MaxPool2d(2, 2), # 28 x 28 -> 14 x 14
            nn.Conv2d(32, 64, 3, padding=1), # 14 x 14 -> 16 x 16 -> 14 x 14
            nn.ReLU(True),
            nn.MaxPool2d(2, 2) # 14 x 14 -> 7 x 7
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(7*7*64, 100),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        output = self.layer(x)
        output = output.view(batch_size, -1) # [batch_size x 7*7*64 x 100]
        output = self.fc_layer(output)

        return output

In [0]:
# Loss and Optimizer

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss() # Q. 6장 RNN에서는 CrossEntropy쓰면 에러가 났었는데...?

# Regularization
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.1)

cuda:0


In [0]:
# Train

for i in range(num_epochs):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
    
    if i % 10 == 0:
        print(loss)

tensor(2.2994, device='cuda:0', grad_fn=<NllLossBackward>)


In [0]:
# Test

correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model(x)
        output_idx = torch.argmax(output, 1) # 아래와 같음. 
        # _,output_idx = torch.max(output,1)

        total += label.size(0)
        correct += (output_idx == y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 10.226362228393555


# Experiment
L1 loss를 적용해보도록 하자

In [0]:
# Loss and Optimizer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model1 = CNN().to(device)
loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model1.parameters(), lr=learning_rate)
c = 1.0

# Train
for i in range(num_epochs):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y_) # scalar임.

        # loss = loss + L1_norm 이므로, L1norm을 구해서 더해줌.
        norm = torch.tensor(0.0).to(device) # torch.cuda.FloatTensor()로 scalar 변형이 안되길래 이렇게 함.

        # 그냥 data만 더할경우, parameter가 아니기 때문에, 학습이 제대로 진행되지 않는다.
        # 따라서 parameters()를 통해서 parameter자체를 norm으로 더해준다.
        for parameter in model1.parameters():
            temp = torch.norm(parameter, p=1)
            norm += temp
        loss += c * norm
        loss.backward()
        optimizer.step()
    
    if i % 10 == 0:
        print(loss)

cuda:0
tensor(128.2100, device='cuda:0', grad_fn=<AddBackward0>)


In [0]:
# Test

correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model1(x)
        output_idx = torch.argmax(output, 1) # 아래와 같음. 
        # _,output_idx = torch.max(output,1)

        total += label.size(0)
        correct += (output_idx == y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 9.805688858032227


## L2 norm도 똑같이 구해보자

In [0]:
# Loss and Optimizer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model2 = CNN().to(device)
loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model2.parameters(), lr=learning_rate)
c = 0.1 # decay값이랑 동일하게 세팅해보자.

# Train
for i in range(num_epochs):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y_) # scalar임.

        # loss = loss + L2_norm 이므로, L2 norm을 구해서 더해주어야 함.
        norm = torch.tensor(0.0).to(device) # torch.cuda.FloatTensor()로 scalar 변형이 안되길래 이렇게 함.

        # 그냥 data만 더할경우, parameter가 아니기 때문에, 학습이 제대로 진행되지 않는다.
        # 따라서 parameters()를 통해서 parameter자체를 norm으로 더해준다.
        for parameter in model2.parameters():
            temp = torch.norm(parameter, p=2)
            norm += temp
        loss += c * norm
        loss.backward()
        optimizer.step()
    
    if i % 10 == 0:
        print(loss)

cuda:0
tensor(4.2307, device='cuda:0', grad_fn=<AddBackward0>)


 기존의 optimizer에서 weight_decay로 넣었던 것보다 loss가 크다. 비슷하게 나올 것으로 기대했는데...

In [0]:
# Test

correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)

        output = model2(x)
        output_idx = torch.argmax(output, 1) # 아래와 같음. 
        # _,output_idx = torch.max(output,1)

        total += label.size(0)
        correct += (output_idx == y_).sum().float()

    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 9.815705299377441


정확도는 **L1 loss, L2 loss, Custom L2 loss** 모두 비슷한 것으로 확인 됨.  
Weight를 한번 비교해보도록 하자.

In [0]:
def check_weight(model, sparse_threshold=0.0005):
    for param in model.parameters():
        # nz = torch.where(param == 0) # weight가 0인 경우는 없었음.
        t = torch.abs(param).max() * sparse_threshold # 따라서 적절한 threshold 밑에 있는 수가 몇인지를 파악해봄.
        nz = torch.where(abs(param) < t)[0].size()[0]
        print(nz)

print('L2 norm')
check_weight(model)
print('L1 norm')
check_weight(model1)
print('Custom L2 norm')
check_weight(model2)

L2 norm
0
0
3
0
15
0
154
0
0
0
L1 norm
0
0
2
0
5
0
171
0
1
0
Custom L2 norm
0
0
4
0
16
0
138
0
0
0


흠 애매하다... 확실하지 않으니 epochs를 늘려서 다시 한번 ㄱ

In [0]:
# Loss and Optimizer
num_epochs = 100 # 200으로 하면 너무 오래걸림.

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# 슬슬 귀찮음.
model = CNN().to(device)
model1 = CNN().to(device)
model2 = CNN().to(device)
models = [(model, None), (model1, 1.0), (model2, 0.1)] # model, p

loss_func = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(
    list(model.parameters())
    + list(model1.parameters())
    + list(model2.parameters()), lr=learning_rate)
# c = 1.0 # 따로 넣어주겠음.

def train(model, x, y_, c, p=None):
    optimizer.zero_grad()
    output = model(x)
    loss = loss_func(output, y_)

    if p == None:
        pass        
    else:
        for parameter in model.parameters():
            temp = torch.norm(parameter, p=p)
            norm += temp

        loss += c * norm
    loss.backward()
    optimizer.step()

# Train
for i in range(num_epochs):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        # 깔끔한 코드를 위해 함수화
        for model, p in models:
            train(model, x, y_, p)
    if i % 10 == 0:
        print(i )

cuda:0
0
10
20
30
40
50
60
70
80
90


In [0]:
# Test

with torch.no_grad():
    for model, _ in models:
        correct = 0
        total = 0

        for image, label in test_loader:
            x = image.to(device)
            y_ = label.to(device)

            output = model(x)
            output_idx = torch.argmax(output, 1) # 아래와 같음. 
            # _,output_idx = torch.max(output,1)

            total += label.size(0)
            correct += (output_idx == y_).sum().float()

        print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 59.675479888916016
Accuracy of Test Data: 42.72836685180664
Accuracy of Test Data: 59.45512771606445


음 어큐러시가 상당히 올라갔네

In [0]:
def check_weight(model, sparse_threshold=0.0005):
    for param in model.parameters():
        # nz = torch.where(param == 0) # weight가 0인 경우는 없었음.
        t = torch.abs(param).max() * sparse_threshold # 따라서 적절한 threshold 밑에 있는 수가 몇인지를 파악해봄.
        nz = torch.where(abs(param) < t)[0].size()[0]
        print(nz)

print('L2 norm')
check_weight(model)
print('L1 norm')
check_weight(model1)
print('Custom L2 norm')
check_weight(model2)

L2 norm
0
0
0
0
10
0
230
0
0
0
L1 norm
0
0
7
0
14
0
256
0
0
0
Custom L2 norm
0
0
0
0
10
0
230
0
0
0


생각만큼 의미있지는 않은듯. 여튼 뭐 줄어드는 것은 확인하였고, custom L2 loss와 동일함을 확인함.